In [ ]:
import os
import sys
from glob import glob
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
#!apt-get -y install libgl1-mesa-glx
import cv2
from PIL import Image
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from time import time
import torch.utils.data as data
from torchvision import transforms  
import matplotlib.pyplot as plt
import seaborn as sns
import torch.optim as optim

In [ ]:
#!pip install albumentations

In [ ]:
!nvidia-smi

In [ ]:
class cfg:
    data_dir = '/opt/ml/input/data/train'
    img_dir = f'{data_dir}/images'
    df_path = f'{data_dir}/train.csv'

In [ ]:
num2class = ['incorrect_mask', 'mask1', 'mask2', 'mask3',
             'mask4', 'mask5', 'normal']
class2num = {k: v for v, k in enumerate(num2class)}

df = pd.read_csv(cfg.df_path)
# df.head()

In [ ]:
# df[(df.age>=60) & (df.gender=="male")]  #60대이상 남자는 83 여자는 109명
# #df[df.age>=60]

In [ ]:
def get_ext(img_dir, img_id):
    """
    학습 데이터셋 이미지 폴더에는 여러 하위폴더로 구성되고, 이 하위폴더들에는 각 사람의 사진들이 들어가있습니다. 하위폴더에 속한 이미지의 확장자를 구하는 함수입니다.
    
    Args:
        img_dir: 학습 데이터셋 이미지 폴더 경로 
        img_id: 학습 데이터셋 하위폴더 이름

    Returns:
        ext: 이미지의 확장자
    """
    filename = os.listdir(os.path.join(img_dir, img_id))[0] #매개변수 path의 모든 디렉터리와 파일을 배열로 받는다. 그중 0번째 idx를 파일이름으로
    ext = os.path.splitext(filename)[-1].lower() #확장자를 뽑으므로 -1번째
    return ext

In [ ]:
def get_img_stats(img_dir, img_ids):
    """
    데이터셋에 있는 이미지들의 크기와 RGB 평균 및 표준편차를 수집하는 함수입니다.
    
    Args:
        img_dir: 학습 데이터셋 이미지 폴더 경로 
        img_ids: 학습 데이터셋 하위폴더 이름들, 각 사람들 ex) 000002_female~

    Returns:
        img_info: 이미지들의 정보 (크기, 평균, 표준편차)
    """
    img_info = dict(heights=[], widths=[], means=[], stds=[])
    for img_id in tqdm(img_ids):
        for path in glob(os.path.join(img_dir, img_id, '*')): #glob로 그 사람폴더내 모든 사진을 배열로 가져옴
            img = np.array(Image.open(path)) #path는 사진 하나를 의미, 그것을 3차원 텐서로 RGB표현함
            h, w, _ = img.shape
            img_info['heights'].append(h)
            img_info['widths'].append(w)
            img_info['means'].append(img.mean(axis=(0,1))) #채널별로 평균과 표준편차를 구한다
            img_info['stds'].append(img.std(axis=(0,1)))
    return img_info
    

In [ ]:
#img_info = get_img_stats(cfg.img_dir, df.path.values)

In [ ]:
# print(f'Total number of people is {len(df)}')
# print(f'Total number of images is {len(df) * 7}')
# print(f'Minimum height for dataset is {np.min(img_info["heights"])}')
# print(f'Maximum height for dataset is {np.max(img_info["heights"])}')
# print(f'Average height for dataset is {int(np.mean(img_info["heights"]))}')
# print(f'Minimum width for dataset is {np.min(img_info["widths"])}')
# print(f'Maximum width for dataset is {np.max(img_info["widths"])}')
# print(f'Average width for dataset is {int(np.mean(img_info["widths"]))}')

# print(f'RGB Mean: {np.mean(img_info["means"], axis=0) / 255.}')
# print(f'RGB Standard Deviation: {np.mean(img_info["stds"], axis=0) / 255.}')

In [ ]:
# plt.figure(figsize=(6, 4.5)) 
# ax = sns.countplot(x = 'gender', data = df, palette=["#55967e", "#263959"])

# plt.xticks( np.arange(2), ['female', 'male'] )
# plt.title('Sex Ratio',fontsize= 14)
# plt.xlabel('')
# plt.ylabel('Number of images')

# counts = df['gender'].value_counts()
# counts_pct = [f'{elem * 100:.2f}%' for elem in counts / counts.sum()]
# for i, v in enumerate(counts_pct):
#     ax.text(i, 0, v, horizontalalignment = 'center', size = 14, color = 'w', fontweight = 'bold')
    
# plt.show()


In [ ]:
# sns.displot(df, x="age", hue="gender", stat="density")

In [ ]:
# img_id = df.iloc[500].path
# ext = get_ext(cfg.img_dir, img_id)

In [ ]:
# plt.figure()
# plt.subplot(111)

# for class_id in num2class:
#     img = np.array(Image.open(os.path.join(cfg.img_dir, img_id, class_id+ext)).convert('L'))
#     #print(img)
#     histogram, bin_edges = np.histogram(img, bins=256, range=(0, 255))
#     sns.lineplot(data=histogram)

# plt.legend(num2class)
# plt.title('Class Grayscale Histogram Plot', fontsize=15)

In [ ]:
def plot_raw_images(img_dir, img_id):
    """
    마스크 미착용 이미지를 시각화하는 함수입니다.
    
    Args:
        img_dir: 학습 데이터셋 이미지 폴더 경로 
        img_id: 학습 데이터셋 하위폴더 이름
    """
    ext = get_ext(img_dir, img_id)
    img = np.array(Image.open(os.path.join(img_dir, img_id, 'normal' + ext)))
    
    plt.figure(figsize=(6,6))
    plt.imshow(img)

In [ ]:
# plot_raw_images(cfg.img_dir, img_id)

In [ ]:
# from torchvision import transforms
# from torchvision.transforms import Resize, ToTensor, Normalize
# class TestDataset(Dataset):
#     def __init__(self, img_paths, transform):
#         self.img_paths = glob(os.path.join(cfg.img_dir, img_paths, '*'))
#         self.transform = transform

#     def __getitem__(self, index):
#         image = Image.open(self.img_paths[index])
#         if self.transform:
#             image = self.transform(image)
#         return image
    
#     def __len__(self):
#         return len(self.img_paths)
    

In [ ]:

# img_id='000001_female_Asian_45'
# img = np.array(Image.open(os.path.join(cfg.img_dir, img_id, 'normal' + ext)))
# #print(img)
# #plt.imshow(img)
# # tsfm=transforms.Compose([
# #      transforms.ToPILImage(),
# #      transforms.ToTensor(),
# #      transforms.Normalize(mean=(0.5,), std=(0.5,))
# # ])
# # img=tsfm(img)
# transform = transforms.Compose([
#     # BILINEAR(2),
#     ToTensor(),
#     Normalize(mean=(0.5, 0.5, 0.5), std=(0.2, 0.2, 0.2)),
#     transforms.ToPILImage(),
# ])
# dataset = TestDataset(img_id, transform)
# #print(dataset.img_paths)
# #/opt/ml/input/data/train/images/000001_female_Asian_45/normal.jpg
# #print(len(dataset))
# #plt.imshow(dataset[0])
# plt.imshow(dataset[1])
# #plt.imshow(dataset)


In [ ]:
mean, std = (0.56019358 ,0.52410121, 0.501457), (0.23318603, 0.24300033 ,0.24567522)

In [ ]:
from albumentations import *
from albumentations.pytorch import ToTensorV2


def get_transforms(need=('train', 'val'), img_size=(512, 384), mean=(0.548, 0.504, 0.479), std=(0.237, 0.247, 0.246)):
    """
    train 혹은 validation의 augmentation 함수를 정의합니다. train은 데이터에 많은 변형을 주어야하지만, validation에는 최소한의 전처리만 주어져야합니다.
    
    Args:
        need: 'train', 혹은 'val' 혹은 둘 다에 대한 augmentation 함수를 얻을 건지에 대한 옵션입니다.
        img_size: Augmentation 이후 얻을 이미지 사이즈입니다.
        mean: 이미지를 Normalize할 때 사용될 RGB 평균값입니다.
        std: 이미지를 Normalize할 때 사용될 RGB 표준편차입니다.

    Returns:
        transformations: Augmentation 함수들이 저장된 dictionary 입니다. transformations['train']은 train 데이터에 대한 augmentation 함수가 있습니다.
    """
    transformations = {}
    if 'train' in need:
        transformations['train'] = Compose([
            Resize(img_size[0], img_size[1], p=1.0),
            HorizontalFlip(p=0.5),
            ShiftScaleRotate(p=0.5),
            HueSaturationValue(hue_shift_limit=0.2, sat_shift_limit=0.2, val_shift_limit=0.2, p=0.5),
            RandomBrightnessContrast(brightness_limit=(-0.1, 0.1), contrast_limit=(-0.1, 0.1), p=0.5),
            GaussNoise(p=0.5),
            Normalize(mean=mean, std=std, max_pixel_value=255.0, p=1.0),
            ToTensorV2(p=1.0),
        ], p=1.0)
    if 'val' in need:
        transformations['val'] = Compose([
            Resize(img_size[0], img_size[1]),
            Normalize(mean=mean, std=std, max_pixel_value=255.0, p=1.0),
            ToTensorV2(p=1.0),
        ], p=1.0)
    return transformations

In [ ]:
class MaskLabels:
    mask = 0
    incorrect = 1
    normal = 2

class GenderLabels:
    male = 0
    female = 1

class AgeGroup:
    map_label = lambda x: 0 if int(x) < 30 else 1 if int(x) < 60 else 2

In [ ]:
class MaskDataset(Dataset):
    num_classes = 3 * 2 * 3

    _file_names = {
        "mask1.jpg": MaskLabels.mask,
        "mask2.jpg": MaskLabels.mask,
        "mask3.jpg": MaskLabels.mask,
        "mask4.jpg": MaskLabels.mask,
        "mask5.jpg": MaskLabels.mask,
        "incorrect_mask.jpg": MaskLabels.incorrect,
        "normal.jpg": MaskLabels.normal
    }

    image_paths = []
    mask_labels = []
    gender_labels = []
    age_labels = []

    def __init__(self, img_dir, transform=None):
        self.img_dir = img_dir
        self.mean = mean
        self.std = std
        self.transform = transform

        self.setup()
    
    def set_transform(self, transform):
        """
        transform 함수를 설정하는 함수입니다.
        """
        self.transform = transform

    def setup(self):
        profiles = os.listdir(self.img_dir)
        for profile in profiles:
            for file_name, label in self._file_names.items():
                img_path = os.path.join(self.img_dir, profile, file_name)  # (resized_data, 000004_male_Asian_54, mask1.jpg)
                if os.path.exists(img_path):
                    self.image_paths.append(img_path)
                    self.mask_labels.append(label)

                    id, gender, race, age = profile.split("_")
                    gender_label = getattr(GenderLabels, gender)
                    age_label = AgeGroup.map_label(age)

                    self.gender_labels.append(gender_label)
                    self.age_labels.append(age_label)


    def __getitem__(self, index):
        """
        데이터를 불러오는 함수입니다. 
        데이터셋 class에 데이터 정보가 저장되어 있고, index를 통해 해당 위치에 있는 데이터 정보를 불러옵니다.
        
        Args:
            index: 불러올 데이터의 인덱스값입니다.
        """
        # 이미지를 불러옵니다.
        image_path = self.image_paths[index]
        image = Image.open(image_path)
        
        # 레이블을 불러옵니다.
        mask_label = self.mask_labels[index]
        gender_label = self.gender_labels[index]
        age_label = self.age_labels[index]
        multi_class_label = mask_label * 6 + gender_label * 3 + age_label
        
        # 이미지를 Augmentation 시킵니다.
        image_transform = self.transform(image=np.array(image))['image']
        return image_transform, multi_class_label

    def __len__(self):
        return len(self.image_paths)
    

In [ ]:
# 정의한 Augmentation 함수와 Dataset 클래스 객체를 생성합니다.
transform = get_transforms(mean=mean, std=std)

dataset = MaskDataset(
    img_dir=cfg.img_dir
)

# train dataset과 validation dataset을 8:2 비율로 나눕니다.
n_val = int(len(dataset) * 0.2)
n_train = len(dataset) - n_val
train_dataset, val_dataset = data.random_split(dataset, [n_train, n_val])

# 각 dataset에 augmentation 함수를 설정합니다.
train_dataset.dataset.set_transform(transform['train'])
val_dataset.dataset.set_transform(transform['val'])

In [ ]:
# training dataloader은 데이터를 섞어주어야 합니다. (shuffle=True)
train_loader = DataLoader(
    train_dataset,
    batch_size=512,
    num_workers=0,
    shuffle=True
)

val_loader = DataLoader(
    val_dataset,
    batch_size=512,
    num_workers=0,
    shuffle=False
)

In [ ]:
# device = torch.cuda.set_device(0)
# images, labels = next(iter(train_loader))
# images=images.to(device)
# labels=labels.to(device)
# print(f'images shape: {images.shape}')
# print(f'labels shape: {labels.shape}')

In [ ]:
# # Augmentation으로 이미지를 Normalize했기 때문에, 역으로 다시 Normalize 해주어야합니다.
# inv_normalize = transforms.Normalize(
#     mean=[-m / s for m, s in zip(mean, std)],
#     std=[1 / s for s in std]
# )

# n_rows, n_cols = 4, 3

# fig, axes = plt.subplots(n_rows, n_cols, sharex=True, sharey=True, figsize=(16, 24))
# for i in range(n_rows*n_cols):
#     axes[i%n_rows][i//(n_cols+1)].imshow(inv_normalize(images[i]).permute(1, 2, 0))
#     axes[i%n_rows][i//(n_cols+1)].set_title(f'Label: {labels[i]}', color='r')
# plt.tight_layout()

In [ ]:
# class Model(nn.Module):
#     def __init__(self):
#         super(Model, self).__init__()
#         self.conv1 = nn.Conv2d(in_channels=1, out_channels=3, kernel_size=3, bias=True)
#         self.bn1 = nn.BatchNorm2d(num_features=3)
#         self.conv2 = nn.Conv2d(in_channels=3, out_channels=5, kernel_size=3, bias=False)

#     def forward(self, x):
#         x = F.relu(self.bn1(self.conv1(x)))
#         return F.relu(self.conv2(x))

In [ ]:
# model = Model()
# model

In [ ]:
# 1. using named_parameters()
# for param, weight in model.named_parameters():
#     print(f"{param:20} - size: {weight.size()}")
#     print(weight)
#     print("-" * 100)
#     print()
#model.state_dict(), model.parameters()

In [ ]:
# 2. directly access with member variable
# print(model.conv1.weight)
# print(model.conv1.bias)

In [ ]:
# ##학습된 모델 저장하기
# save_folder = "/opt/ml/code/save_model"
# save_path = os.path.join(save_folder, "best.pth")   # ./runs/best.pth
# os.makedirs(save_folder, exist_ok=True)  

# torch.save(model.state_dict(), save_path)
# print(f"Model saving success at {save_path}")
# print(f"Saved models : {os.listdir(save_folder)}")

In [ ]:
# new_model = Model()
# new_model.load_state_dict(torch.load(save_path))
# print(f"Model loading success from {save_path}")

In [ ]:
# for (name, trained_weight), (_, saved_weight) in zip(model.named_parameters(), new_model.named_parameters()):
#     is_equal = torch.equal(trained_weight, saved_weight)
#     print(f"parameter {name:15} from trained model and loaded model is equal? -> {is_equal}")

In [ ]:
print(torch.version)

In [ ]:
from torchvision.models import resnet50
# device=torch.device('cuda')
model = resnet50(pretrained=True)
num_classes = 18
model.fc = nn.Linear(2048,18)
# Freeze except fc parts
model.conv1.requires_grad_(False)
model.bn1.requires_grad_(False)
model.layer1.requires_grad_(False)
model.layer1.requires_grad_(False)
model.layer2.requires_grad_(False)
model.layer3.requires_grad_(False)
model.layer4.requires_grad_(False)
for param, weight in model.named_parameters():
    print(f"param {param:20} required gradient? -> {weight.requires_grad}")
model.cuda()
#model

In [ ]:
import torch.nn.init as init

def initialize_weights(model):
    """
    Initialize all weights using xavier uniform. 
    For more weight initialization methods, check https://pytorch.org/docs/stable/nn.init.html
    """
    for m in model.modules():
        if isinstance(m, nn.Conv2d):
            init.xavier_uniform_(m.weight.data)
            if m.bias is not None:
                m.bias.data.zero_()
        elif isinstance(m, nn.BatchNorm2d):
            m.weight.data.fill_(1)
            m.bias.data.zero_()
        elif isinstance(m, nn.Linear):
            m.weight.data.normal_(0, 0.01)
            m.bias.data.zero_()

In [ ]:
model.fc.weight.data.normal_(0,0.01)
model.fc.weight.data.zero_()

In [ ]:
optimizer=optim.Adam(model.parameters(),lr=3e-4)
criterion=nn.CrossEntropyLoss()
model.train()

In [ ]:
for epoch in range(10):
    running_loss=0.0
    for i,data in tqdm(enumerate(train_loader,0)):
        inputs,labels=data
        inputs = inputs.cuda()
        labels = labels.cuda()
        optimizer.zero_grad()
        
        outputs=model(inputs)
        loss=criterion(outputs,labels)
        loss.backward()
        optimizer.step()

        running_loss+=loss.item()
#         if i%256==255:
#             print('[%d, %5d] loss : %.3f' % (epoch+1,i+1,running_loss/2000))
#             running_loss=0.0
        print('[%d, %5d] loss : %.3f' % (epoch+1,i+1,running_loss))
        running_loss=0.0
print("finished training")



In [ ]:
torch.cuda.is_available()

In [ ]:

##학습된 모델 저장하기
save_folder = "/opt/ml/code/save_model"
save_path = os.path.join(save_folder, "best2.pth")   # ./runs/best.pth
os.makedirs(save_folder, exist_ok=True)  

torch.save(model.state_dict(), save_path)
print(f"Model saving success at {save_path}")
print(f"Saved models : {os.listdir(save_folder)}")

In [ ]:
model.state_dict()

In [ ]:
# with torch.no_grad():
#         print("Calculating validation results...")
#         model.eval()
#         val_loss_items = []
#         val_acc_items = []
#         for val_batch in val_loader:
#             inputs, labels = val_batch
#             inputs = inputs.cuda()
#             labels = labels.cuda()

#             outs = model(inputs)
#             preds = torch.argmax(outs, dim=-1)

#             loss_item = criterion(outs, labels).item()
#             acc_item = (labels == preds).sum().item()
#             val_loss_items.append(loss_item)
#             val_acc_items.append(acc_item)

#         val_loss = np.sum(val_loss_items) / len(val_loader)
#         val_acc = np.sum(val_acc_items) / len(val_loader)
#         print(
#             f"[Val] acc : {val_acc:4.2%}, loss: {val_loss:4.2} || "
#         )

In [ ]:
from torchvision import transforms
from torchvision.transforms import Resize, ToTensor, Normalize
class TestDataset(Dataset):
    def __init__(self, img_paths, transform):
        self.img_paths = img_paths
        self.transform = transform

    def __getitem__(self, index):
        image = Image.open(self.img_paths[index])

        if self.transform:
            image = self.transform(image)
        return image

    def __len__(self):
        return len(self.img_paths)

In [ ]:
# meta 데이터와 이미지 경로를 불러옵니다.
test_dir='/opt/ml/input/data/eval'
submission = pd.read_csv(os.path.join(test_dir, 'info.csv'))
image_dir = os.path.join(test_dir, 'images')
# Test Dataset 클래스 객체를 생성하고 DataLoader를 만듭니다.
image_paths = [os.path.join(image_dir, img_id) for img_id in submission.ImageID]
transform = transforms.Compose([
    Resize((512, 384), Image.BILINEAR),
    ToTensor(),
    Normalize(mean=mean, std=std),
])
dataset = TestDataset(image_paths, transform)

loader = DataLoader(
    dataset,
    shuffle=False
)

In [ ]:
# 모델을 정의합니다. (학습한 모델이 있다면 torch.load로 모델을 불러주세요!)
model_root = '/opt/ml/code/save_model/best1.pth'
from torchvision.models import resnet50
model = resnet50(pretrained=True)
model.to(device)
num_classes = 18
model.fc = nn.Linear(2048,18)
model.load_state_dict(torch.load(model_root))
model.eval()


In [ ]:
model.state_dict()

In [ ]:
# 모델이 테스트 데이터셋을 예측하고 결과를 저장합니다.
all_predictions = []
for images in loader:
    with torch.no_grad():
        images = images.cuda()
        pred = model(images)
        pred = pred.argmax(dim=-1)
        all_predictions.extend(pred.cpu().numpy())
submission['ans'] = all_predictions

# 제출할 파일을 저장합니다.
submission.to_csv(os.path.join(test_dir, 'submission3.csv'), index=False)
print('test inference is done!')